In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
dt = pd.read_csv('data\Dataset.csv')

C:\Users\MoorthyMitturu\AppData\Local\Temp\ipykernel_11036\112541974.py:1: DtypeWarning: Columns (1,7,8,16,17,18,19,20,35) have mixed types. Specify dtype option on import or set low_memory=False.
  dt = pd.read_csv('data\Dataset.csv')


In [3]:
# 2. Check for mixed types in numeric columns
for col in dt.select_dtypes(include='object').columns:
    print(f"{col}: {dt[col].unique()[:10]}")

Client_Income: ['6750' '20250' '18000' '15750' '33750' '11250' '13500' '12150' '27000'
 '8100']
Credit_Amount: ['61190.55' '15282' '59527.35' '53870.4' '133988.4' '13752' '128835'
 '60415.2' '45000' '16320.15']
Loan_Annuity: ['3416.85' '1826.55' '2788.2' '2295.45' '3547.35' '653.85' '3779.55'
 '3097.8' '1200.15' '1294.65']
Accompany_Client: ['Alone' 'Relative' 'Others' 'Kids' nan 'Partner' '##' 'Group']
Client_Income_Type: ['Commercial' 'Service' 'Retired' 'Govt Job' nan 'Student' 'Unemployed'
 'Maternity leave' 'Businessman']
Client_Education: ['Secondary' 'Graduation' 'Graduation dropout' nan 'Junior secondary'
 'Post Grad']
Client_Marital_Status: ['M' 'W' 'S' nan 'D']
Client_Gender: ['Male' 'Female' nan 'XNA']
Loan_Contract_Type: ['CL' 'RL' nan]
Client_Housing_Type: ['Home' 'Family' 'Office' 'Municipal' nan 'Rental' 'Shared']
Population_Region_Relative: ['0.028663' '0.008575' '0.0228' '0.010556' '0.020713' '0.019101'
 '0.016612' '0.009175' '0.006008' '0.001417']
Age_Days: ['13957' '

In [4]:
# Define columns that should be numeric
numeric_cols = [
    'Client_Income', 'Credit_Amount', 'Loan_Annuity', 'Population_Region_Relative',
    'Age_Days', 'Employed_Days', 'Registration_Days', 'ID_Days', 'Score_Source_3'
]
# Convert numeric columns (force errors to NaN for cleaning)
for col in numeric_cols:
    dt[col] = pd.to_numeric(dt[col], errors='coerce')

In [5]:
# Standardize categorical 'Yes/No' tags
dt['Client_Permanent_Match_Tag'] = dt['Client_Permanent_Match_Tag'].map({'Yes': 1, 'No': 0})
dt['Client_Contact_Work_Tag'] = dt['Client_Contact_Work_Tag'].map({'Yes': 1, 'No': 0})

# Handle categorical anomalies and placeholders
dt['Accompany_Client'] = dt['Accompany_Client'].replace('##', np.nan)
dt['Client_Gender'] = dt['Client_Gender'].replace('XNA', np.nan)
dt['Type_Organization'] = dt['Type_Organization'].replace('XNA', np.nan)

# Convert categorical columns to category dtype for memory efficiency
categorical_cols = [
    'Accompany_Client', 'Client_Income_Type', 'Client_Education', 'Client_Marital_Status',
    'Client_Gender', 'Loan_Contract_Type', 'Client_Housing_Type', 'Client_Occupation',
    'Type_Organization'
]
for col in categorical_cols:
    dt[col] = dt[col].astype('category')
    
# Sentinel handling: replace '365243' in Employed_Days (means missing/unemployed)
dt['Employed_Days'] = dt['Employed_Days'].replace(365243, np.nan)

In [6]:
# Check for mixed types in numeric columns
for col in dt.select_dtypes(include='object').columns:
    print(f"{col}: {dt[col].unique()[:10]}")

#### High-Level Strategy

The preprocessing pipeline will have four layers:

Missing value imputation — numerical: median; categorical: mode or "Unknown".

Feature creation — business logic–based new features (ratios, age, stability, etc.).

Encoding categorical variables — one-hot for low-cardinality features; target or frequency encoding for high-cardinality ones.

Scaling and final cleaning — numeric standardization where necessary (only for models sensitive to scale, e.g. logistic regression, neural nets).

🧩 Step 2 — Business-Driven Feature Engineering

Based on EDA findings, we’ll create meaningful engineered variables:
| New Feature            | Formula                           | Business Rationale                                                         |
| ---------------------- | --------------------------------- | -------------------------------------------------------------------------- |
| `Age_Years`            | `(-Age_Days / 365).round(1)`      | Converts days to age; older borrowers tend to default less.                |
| `Employment_Years`     | `(-Employed_Days / 365).round(1)` | Captures stability; long employment reduces risk.                          |
| `Credit_Income_Ratio`  | `Credit_Amount / Client_Income`   | Measures affordability; high ratios → higher risk.                         |
| `Annuity_Income_Ratio` | `Loan_Annuity / Client_Income`    | Monthly repayment pressure relative to income.                             |
| `Credit_Annuity_Ratio` | `Credit_Amount / Loan_Annuity`    | Effective repayment duration; signals loan structure.                      |
| `Missing_Count`        | Count of NaN values per row       | Captures data sparsity; high missingness might correlate with higher risk. |


#### Numeric imputation

In [7]:
num_impute_cols = ['Client_Income', 'Credit_Amount', 'Loan_Annuity', 
                   'Population_Region_Relative', 'Age_Days', 'Employed_Days']
for col in num_impute_cols:
    dt[col + '_missing'] = dt[col].isnull().astype(int)
    dt[col].fillna(dt[col].median(), inplace=True)


C:\Users\MoorthyMitturu\AppData\Local\Temp\ipykernel_11036\908452885.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dt[col].fillna(dt[col].median(), inplace=True)
C:\Users\MoorthyMitturu\AppData\Local\Temp\ipykernel_11036\908452885.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

#### Categorical imputation

In [8]:
cat_impute_cols = ['Client_Income_Type', 'Client_Education', 
                   'Client_Marital_Status', 'Client_Gender', 
                   'Client_Occupation', 'Type_Organization']
for col in cat_impute_cols:
    dt[col] = dt[col].astype('category')
    dt[col] = dt[col].cat.add_categories('Unknown').fillna('Unknown')


#### Feature creation

In [ ]:
dt['Age_Years'] = (-dt['Age_Days'] / 365).round(1)
dt['Employment_Years'] = (-dt['Employed_Days'] / 365).round(1)
dt['Credit_Income_Ratio'] = dt['Credit_Amount'] / (dt['Client_Income'] + 1)
dt['Annuity_Income_Ratio'] = dt['Loan_Annuity'] / (dt['Client_Income'] + 1)
dt['Credit_Annuity_Ratio'] = dt['Credit_Amount'] / (dt['Loan_Annuity'] + 1)
dt['Missing_Count'] = dt.isnull().sum(axis=1)

In [11]:
dt['Loan_Contract_Type'].value_counts()

Loan_Contract_Type
CL    107118
RL     11087
Name: count, dtype: int64

In [12]:
dt['Client_Marital_Status'].value_counts()

Client_Marital_Status
M          87349
S          17404
D           7556
W           6074
Unknown     3473
Name: count, dtype: int64

#### Encoding categorical variables

For low-cardinality (e.g., Client_Gender, Loan_Contract_Type): one-hot encode.

For medium-to-high cardinality (e.g., Client_Occupation, Type_Organization): frequency encode to keep model size manageable.

In [13]:
low_card = ['Client_Gender', 'Loan_Contract_Type', 'Client_Marital_Status']
dt = pd.get_dummies(dt, columns=low_card, drop_first=True)

high_card = ['Client_Occupation', 'Type_Organization']
for col in high_card:
    freq_map = dt[col].value_counts(normalize=True)
    dt[col] = dt[col].map(freq_map)


#### Final scaling (if using logistic regression or NN)

In [14]:
from sklearn.preprocessing import StandardScaler
scale_cols = ['Client_Income', 'Credit_Amount', 'Loan_Annuity',
              'Credit_Income_Ratio', 'Annuity_Income_Ratio', 'Credit_Annuity_Ratio']
scaler = StandardScaler()
dt[scale_cols] = scaler.fit_transform(dt[scale_cols])


In [15]:
dt.shape

(121856, 56)

In [16]:
dt.head(3)

,ID,Client_Income,Car_Owned,Bike_Owned,Active_Loan,House_Own,Child_Count,Credit_Amount,Loan_Annuity,Accompany_Client,...,Annuity_Income_Ratio,Credit_Annuity_Ratio,Missing_Count,Client_Gender_Male,Client_Gender_Unknown,Loan_Contract_Type_RL,Client_Marital_Status_M,Client_Marital_Status_S,Client_Marital_Status_W,Client_Marital_Status_Unknown
0,12142509,-0.882958,0.0,0.0,1.0,0.0,0.0,0.034994,0.491575,Alone,...,3.401764,-0.448770,3,True,False,False,True,False,False,False
1,12138936,0.304060,1.0,0.0,1.0,NaN,0.0,-1.119391,-0.618288,Alone,...,-0.964351,-1.551682,5,True,False,False,True,False,False,False
2,12181264,0.106223,0.0,0.0,1.0,0.0,1.0,-0.006827,0.052843,Alone,...,-0.285223,-0.051256,4,True,False,False,False,False,True,False


In [17]:
dt.columns

Index(['ID', 'Client_Income', 'Car_Owned', 'Bike_Owned', 'Active_Loan',
       'House_Own', 'Child_Count', 'Credit_Amount', 'Loan_Annuity',
       'Accompany_Client', 'Client_Income_Type', 'Client_Education',
       'Client_Housing_Type', 'Population_Region_Relative', 'Age_Days',
       'Employed_Days', 'Registration_Days', 'ID_Days', 'Own_House_Age',
       'Mobile_Tag', 'Homephone_Tag', 'Workphone_Working', 'Client_Occupation',
       'Client_Family_Members', 'Cleint_City_Rating',
       'Application_Process_Day', 'Application_Process_Hour',
       'Client_Permanent_Match_Tag', 'Client_Contact_Work_Tag',
       'Type_Organization', 'Score_Source_1', 'Score_Source_2',
       'Score_Source_3', 'Social_Circle_Default', 'Phone_Change',
       'Credit_Bureau', 'Default', 'Client_Income_missing',
       'Credit_Amount_missing', 'Loan_Annuity_missing',
       'Population_Region_Relative_missing', 'Age_Days_missing',
       'Employed_Days_missing', 'Age_Years', 'Employment_Years',
       'Cr

In [20]:
dt.isnull().mean().mul(100).reset_index()

,index,0
0,ID,0.000000
1,Client_Income,0.000000
2,Car_Owned,2.938715
3,Bike_Owned,2.974002
4,Active_Loan,2.983029
5,House_Own,3.004366
6,Child_Count,2.985491
7,Credit_Amount,0.000000
8,Loan_Annuity,0.000000
9,Accompany_Client,1.442686
